# Visualización de la atención en los transformers

In [1]:
!nvidia-smi

Sun Apr 18 19:41:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000001:00:00.0 Off |                    0 |
| N/A   83C    P0   142W / 149W |   9367MiB / 11441MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           On   | 00000002:00:00.0 Off |                    0 |
| N/A   

In [2]:
%%capture 
!pip install transformers -U
#!pip install pytorch

In [3]:
from sklearn.model_selection import train_test_split
import torch

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
%%capture
!pip install bertviz

In [183]:
import pandas as pd 

test_data = pd.read_csv('amazonEN_test.csv')
test_data_chatbot = pd.read_csv('chatbot_testES.csv')

In [313]:
# Feature extra 2 : reviews length
import regex as re
test_data['length'] = test_data['review_body'].apply(lambda x : len(re.split('\W+',x)))
test_data_chatbot['length'] = test_data_chatbot['Query'].apply(lambda x : len(re.split('\W+',x)))

In [291]:
test_rating = []
for i in range(0,5):
    mask = (test_data['length']< 15) & (test_data['star_rating'] == i)
    test_rat = test_data[mask].reset_index()
    test_rat= test_rat.iloc[0].review_body
    test_rating.append(test_rat)

In [353]:
#test_rating

In [300]:
test_cats = []
    
for i in range(0,30):
    mask = (test_data['length']< 36) & (test_data['product_category_cod'] == i)
    test_cat = test_data[mask].reset_index()
    test_cat = test_cat.iloc[0].review_body
    test_cats.append(test_cat)

In [354]:
#test_cats

In [330]:
test_cats_chatbot = []
for i in range(0,30):
    mask = (test_data_chatbot['length']< 48) & (test_data_chatbot['category_cod'] == i)
    test_cat = test_data_chatbot[mask].reset_index()
    test_cat = test_cat.iloc[0].Query
    test_cats_chatbot.append(test_cat)

In [355]:
#test_cats_chatbot

## BERT (monolingüe)

In [332]:
from transformers import BertModel, BertConfig, BertTokenizer
from transformers import BertForSequenceClassification

# Elegimos el modelo entrenado (5)

#Rating
tokenizer = BertTokenizer.from_pretrained('Bert_body')
#tokenizer = BertTokenizer.from_pretrained('BERT_text_title')
#tokenizer = BertTokenizer.from_pretrained('BERT_text_title_cat')

#Category


#Chatbot


In [333]:
from transformers import BertForSequenceClassification
RANDOM_SEED=8
MAX_LENGTH=260

class_names = ['0','1','2','3','4'] 

model = BertForSequenceClassification.from_pretrained(
    'Bert_body',
    num_labels = len(class_names),  # The number of output labels--2 for binary classification.
    # Esta vez devolvemos el vector de atención =True 
    output_attentions = True,       # Whether the model returns attentions weights.
    output_hidden_states = False ,  # Whether the model returns all hidden-states.   
    vocab_size=tokenizer.vocab_size)

In [350]:
#Elegimos alguna de las frases de test

inputs = tokenizer.encode(test_rating[0], return_tensors='pt',add_special_tokens=True)
outputs = model(inputs)
attention = outputs[-1]  # Output includes attention weights when output_attentions=True
tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 

In [352]:
#Visualizamos la atención 
#Cada color representa una de las 12 cabezas de atención por cada capa del modelo BERT 

from bertviz import head_view,model_view
model_view(attention, tokens)

<IPython.core.display.Javascript object>

## DistilBERT (monolingüe)

In [ ]:
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer
from transformers import DistilBertForSequenceClassification

# Elegimos el modelo entrenado (5)

#Rating
#tokenizer = BertTokenizer.from_pretrained('Chek1mv_body')
tokenizer = DistilBertTokenizer.from_pretrained('Chek1mv_body_title')
#tokenizer = BertTokenizer.from_pretrained('Chek1mv_body_title_cat')

#Category


#Chatbot


In [ ]:
from transformers import BertForSequenceClassification
RANDOM_SEED=8
MAX_LENGTH=260

class_names = ['0','1','2','3','4'] 

model = DistilBertForSequenceClassification.from_pretrained(
    'Chek1mv_body_title',
    num_labels = len(class_names),  # The number of output labels--2 for binary classification.
    # Esta vez devolvemos el vector de atención =True 
    output_attentions = True,       # Whether the model returns attentions weights.
    output_hidden_states = False ,  # Whether the model returns all hidden-states.   
    vocab_size=tokenizer.vocab_size)

In [ ]:
#Elegimos alguna de las frases de test

inputs = tokenizer.encode("These did not work at all. The batteries neither came with a charge on them or would take a charge. Terrible product", return_tensors='pt')
outputs = model(inputs)
attention = outputs[-1]  # Output includes attention weights when output_attentions=True
tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 

In [ ]:
#Visualizamos la atención 
#Cada color representa una de las 12 capas de atención del modelo BERT 

from bertviz import head_view,model_view
head_view(attention, tokens)

# mBERT (multilingüe)

## Few-shot

In [ ]:
# Amazon - Entrenado en inglés + 1000 idioma objetivo , testeamos en español / francés / alemán 

In [ ]:
# Chatbot - Entrenado en español + ingles , testeamos en ingles 

## Zero-shot

In [ ]:
# Amazon - Entrenado en inglés, testeamos en español / francés / alemán 

In [ ]:
# Chatbot - Entrenado en español , testeamos en ingles 